# Importing

In [1]:
import numpy as np
import matplotlib.pyplot as mpl

# Definfing Variables

In [2]:
StartingTempMol = 283.15 #change this to Temp we are using
AirTemp= 277.55 # Air temperature in Kelvin
EndDistance= 135 # End distance in meters
TankRadi= 13.5 # Tank radius in meters
TankHeight= 15 # Tank height in meters
StartingVelMol= 100 #change this to velocity we are using
density = 1415 # Density in kg/m^3

# Initial conditions

In [3]:
# Spatial variables
N=100000 # Number of spatial steps
a=EndDistance/(N-1) # Size of a spatial node
indexToPos = np.arange(0,135+a,a) #Converts a 0-based index to a position in meters
def posToIndex(i): #Converts a position in meters to a 0-based index
    return np.floor(i/a)

# Temporal variables
h=0.01 # Time step in seconds

# Tracking position
hasMolasses = np.zeros(N,dtype=bool) # contains true if there is molasses at a given 
                                     # point, false otherwise
hasMolasses[0] = True # Initially, there is only molasses at the hole

# Heat equation
temp = np.full(N, AirTemp, dtype=np.float64) # Stores the temperature at the given index
temp[0] = StartingTempMol # At the hole, the molasses is the same temperature as in the tank

# Viscosity equation
visc = np.zeros(N, dtype=np.float64)

# Velocity equation
vel = np.zeros(N, dtype=np.float64)
vel[0] = StartingVelMol

# Tracking position

In [4]:
MAX_IND = 0 # Stores the maximum index that is filled with molasses. Will only be valid
            # in 1D.

def xStep():
    
    v = vel[MAX_IND]
    dx = v*h
    dind = posToIndex(dx)
    for i in range(MAX_IND+1, MAX_IND + 1 + dind):
        hasMolasses[i] = True
    MAX_IND = MAX_IND + dind
    



# Heat Equation

In [5]:
c = h/(a*a)

def alpha(T):
    k = 1.3e-3*T + 0.403
    return k/(2.56e3*1415)

def HeatStep():
    alphaarr = alpha(temp)
    Tp = np.zeros(N, dtype=np.float64)
    Tp[0] = StartingTempMol
    Tp[1:MAX_IND] = temp[1:MAX_IND] + c*alphaarr*(temp[2:MAX_IND+1] + temp[:MAX_IND-1] - 2*temp[1:MAX_IND])
    temp = Tp


def HeatEQ (q,t):
    return "hello" #fill in with equation

# Viscosity Equation

In [6]:
E0 = 53.074
R = 8.314
mu0 = 2.3e-9

def ViscEQ(T):
    return mu0 * np.exp(-E0/(R*T))

# Equation of Motion

In [ ]:
# Note: Confined to 1 dimension, velocity is a scalar quantity, and gravity isn't a consideration

MAX_IND = 0

def velStep(): # calculates the velocity at any position within any one timestep 
    muarr = ViscEQ(temp)
    velp = np.zeros(N, dtype=np.float64)
    velp[0] = StartingVelMol
    velp[1:MAX_IND] = vel[1:MAX_IND] + (-muarr*c)*(vel[2:MAX_IND+1] + vel[:MAX_IND-1] - 2*vel[1:MAX_IND])/density
    vel = velp
    

# The stuff below is experimental
# Trying to define a function that produces a 2D array of velocity values
# With each row representing velocity values at a particular timestep
# and each column representing a specific value in the simulation
# (i.e. all column 1 entries are at x meters, column 2 is at x + a, etc.)
# If this works we can do it for viscosity and temperature as well

def velMap(duration):
    dt = h

    t = 0

    velBox = [vel] # array of velocity arrays: each row is one timestep

    while t < duration:
        
        xStep()

        if MAX_IND > 2:
            newvel = velStep()
            velBox += [vel]
        
        t += dt
    
    return velBox
mpl.imshow(velMap(1),aspect="auto")
mpl.show()
    


UnboundLocalError: cannot access local variable 'MAX_IND' where it is not associated with a value

In [ ]:
'''def velMap(duration):
    dt = h

    t = 0

    velBox = [vel] # array of velocity arrays: each row is one timestep

    while t < duration:
        newvel = velStep()
        velBox += [vel]
        t += dt
    
    return velBox
mpl.imshow(velMap(1))
mpl.show()'''